In [1]:
import sys
sys.path.append("/work/multi_doc_analyzer")
sys.path.append("/work/relation_extraction/Bert_model/bert_modify_seq/data/")

import torch as T
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.cuda
from allennlp.nn import util as nn_util
from multi_doc_analyzer.structure.structure import *
from multi_doc_analyzer.tokenization.tokenizer import MDATokenizer
from tqdm import tqdm
from stanfordcorenlp import StanfordCoreNLP

from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.tokenizers import Token
from allennlp.data.token_indexers import TokenIndexer

from allennlp.data.instance import Instance
from allennlp.data.fields import TextField, LabelField, ArrayField

from tacred_preprocess import DataLoader

from allennlp.data.vocabulary import Vocabulary
from allennlp.data.iterators import BucketIterator, DataIterator, BasicIterator
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder
import random

from allennlp.data.token_indexers import PretrainedBertIndexer
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

from sklearn.metrics import precision_recall_fscore_support as prs
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import csv

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [2]:
train_path = "/work/tacred/data/json/train.json"
test_path = "/work/tacred/data/json/test.json"
model_folder = "/work/model_checkpoint/bert_model_checkpoint/bert_modify_seq/tacred/"
output_path = "/work/relation_extraction/Bert_model/bert_modify_seq/analysis/tacred/"

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    seed=1,
    batch_size=200,
    lr=3e-4,                # learning rate
    epochs=100,
    mlp_hidden_sz=300,
    lstm_hidden_sz=768,
    arg_sz=20,              # position embedding size
    max_seq_len=400,
    cuda=T.device("cuda:0")
)

In [4]:
USE_GPU = T.cuda.is_available()
USE_GPU

True

In [5]:
# set seed for both CPU and CUDA
T.manual_seed(config.seed)

In [6]:
e_label2idx = {'PERSON': 0, 'ORGANIZATION': 1, 'DATE': 2, 'NUMBER': 3, 'TITLE': 4, 'COUNTRY': 5, 'LOCATION': 6, 'CITY': 7, 'MISC': 8, 'STATE_OR_PROVINCE': 9, 'DURATION': 10, 'NATIONALITY': 11, 'CAUSE_OF_DEATH': 12, 'CRIMINAL_CHARGE': 13, 'RELIGION': 14, 'URL': 15, 'IDEOLOGY': 16}

r_label2idx = {'no_relation': 0, 'per:title': 1, 'org:top_members/employees': 2, 'per:employee_of': 3, 'org:alternate_names': 4, 'org:country_of_headquarters': 5, 'per:countries_of_residence': 6, 'org:city_of_headquarters': 7, 'per:cities_of_residence': 8, 'per:age': 9, 'per:stateorprovinces_of_residence': 10, 'per:origin': 11, 'org:subsidiaries': 12, 'org:parents': 13, 'per:spouse': 14, 'org:stateorprovince_of_headquarters': 15, 'per:children': 16, 'per:other_family': 17, 'per:alternate_names': 18, 'org:members': 19, 'per:siblings': 20, 'per:schools_attended': 21, 'per:parents': 22, 'per:date_of_death': 23, 'org:member_of': 24, 'org:founded_by': 25, 'org:website': 26, 'per:cause_of_death': 27, 'org:political/religious_affiliation': 28, 'org:founded': 29, 'per:city_of_death': 30, 'org:shareholders': 31, 'org:number_of_employees/members': 32, 'per:date_of_birth': 33, 'per:city_of_birth': 34, 'per:charges': 35, 'per:stateorprovince_of_death': 36, 'per:religion': 37, 'per:stateorprovince_of_birth': 38, 'per:country_of_birth': 39, 'org:dissolved': 40, 'per:country_of_death': 41}


r_idx2label = {v: k for k, v in r_label2idx.items()}

class RelationDatasetReader(DatasetReader):
    """
    Reads Structure object formatted datasets files, and creates AllenNLP instances.
    """
    def __init__(self, token_indexers: Dict[str, TokenIndexer]=None, 
                 MAX_WORDPIECES: int=config.max_seq_len, 
                 is_training = False, reader: DataLoader=None):
        # make sure results may be reproduced when sampling...
        super().__init__(lazy=False)
        random.seed(0)
        self.is_training = is_training
        self.reader = reader
        
        # NOTE AllenNLP automatically adds [CLS] and [SEP] word peices in the begining and end of the context,
        # therefore we need to subtract 2
        self.MAX_WORDPIECES = MAX_WORDPIECES - 2
        
        # BERT specific init
        self._token_indexers = token_indexers

    def text_to_instance(self, sentence: Sentence) -> Instance:

        field = {}
        
        # tokens
        sentence_field = TextField(sentence["tokens"], self._token_indexers)
        fields = {"tokens": sentence_field}

#         arg_vec = T.tensor([[0, 0] for i in range(len(sentence.tokens) + 2)], dtype=T.long)   # long type to feed into embedding layer
        arg_vec = T.tensor([[config.max_seq_len-1, config.max_seq_len-1] for i in range(config.max_seq_len)], dtype=T.long)   # long type to feed into embedding layer

        # position 
        for i in range(sentence["len"]):
            arg_vec[i][0] = sentence["s_position"][i]    # arg_l position, i-1 for [CLS]
            arg_vec[i][1] = sentence["o_position"][i]   # arg_r position, i-1 for [CLS]
        fields["arg_idx"] = ArrayField(arg_vec)

        # information for each sentence length
        fields["sen_len"] = LabelField(sentence["len"], skip_indexing=True)

        # relation
        if self.is_training:
            fields["label"] = LabelField(sentence["relation"], skip_indexing=True)
        
        return Instance(fields)
    
    def _read(self, file_path: str)->Iterator: 
        sen_list = self.reader.data
        for s in sen_list:
            if s["len"] <= config.max_seq_len:
                yield self.text_to_instance(s)

In [7]:
class BERT(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                out_sz: int=len(r_label2idx)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self._position_embeddings = T.nn.Embedding(num_embeddings=(config.max_seq_len)*2, embedding_dim=config.arg_sz, padding_idx=config.max_seq_len*2 - 1)
        
        # bert output is of dimension 768
        self.lstm = T.nn.LSTM(input_size=768 + 2*config.arg_sz, hidden_size=768, batch_first=True, bidirectional=True)
        self.projection1 = nn.Linear(config.lstm_hidden_sz * 2, config.mlp_hidden_sz)
        self.projection2 = nn.Linear(config.mlp_hidden_sz, out_sz)
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self, tokens: Dict[str, T.tensor], arg_idx: T.tensor, sen_len: T.tensor, label: T.tensor = None) -> Dict[str, T.tensor]:
        
        # Bert as embedding
        embeddings = self.word_embeddings(tokens)
        
        # prepare for concatenate
        arg_idx = arg_idx.type(T.long)
        
        # cut both position and token embedding to precise length
        cut_len = max(i for i in sen_len)
        embeddings = embeddings[:,:cut_len,:]
        arg_idx = arg_idx[:,:cut_len,:]
        
        # mask of the extra part of token embedding after cut from cut_len 
        mask = T.tensor([[[1] if j < sen_len[i] else [0] for j in range(cut_len)] for i in range(len(label))], dtype = T.float, device=config.cuda)
        m_embeddings = T.mul(embeddings, mask)
        
        # cancatenate word and position embedding
        arg_emb = self._position_embeddings(arg_idx)
        arg_cat = T.cat((arg_emb[:,:,0,:], arg_emb[:,:,1,:]), -1)
        concat = T.cat((m_embeddings, arg_cat), -1)

        # BiLSTM layer
        ot, hs = self.lstm(concat)

        # MLP
        mlp_hs = self.projection1(T.cat((ot[:, 0, :config.lstm_hidden_sz], ot[:, -1, config.lstm_hidden_sz:]), -1))
        class_logits = self.projection2(mlp_hs)

        # Cross entropy
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

In [8]:
from scipy.special import expit # the sigmoid function
def tonp(tsr): return tsr.detach().cpu().numpy()

In [9]:
# Predict
class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator, total=self.iterator.get_num_batches(ds))
        preds = []
        with T.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [10]:
def plot_comfusion_matrix(label_classes, predict_classes, out_folder, file_name):
    label_types = list(r_idx2label.values())

    cm = confusion_matrix(label_classes, predict_classes, label_types)
    print(cm)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(cm)
    for (i, j), z in np.ndenumerate(cm):
        ax.text(j, i, '{:0.0f}'.format(z), ha='center', va='center', color='white')
    fig.colorbar(cax)
    ax.set_xticklabels([''] + label_types)
    ax.set_yticklabels([''] + label_types)
    plt.xlabel('Predicted')
    plt.ylabel('True')

    plt.savefig(out_folder + 'confusion_matrix_' + file_name + '.png')
    plt.show()
    
    # remove the data which is none or predict none
    
    pre, recall, f1, sup = prs(label_classes, predict_classes, average='macro')
    
    print("Accuracy:", sum(cm[i][i] for i in range(len(cm))) / len(label_classes))
    print("Precision:", pre)
    print("Recall:", recall)
    print("F1 score:", f1)

In [11]:
def err_analyze(ds, true, pred, opt):
    
    # classify different kinds of error
    detail = [[[] for j in range(len(r_label2idx))] for i in range(len(r_label2idx))]
    for i in range(len(ds)):
         if true[i] != pred[i]:
            detail[r_label2idx[true[i]]][r_label2idx[pred[i]]].append(i)
    
    # print into a csv file
    with open(output_path + "error_detail_" + opt + ".csv", "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Sentence", "Two_Entity", "Predict", "Label", "idx"])
        for j in range(len(detail)):
            for k in range(len(detail)):
                with_element = 0
                if k == j:
                    continue
                for i in detail[j][k]:
                    with_element = 1
                    ent1 = []
                    ent2 = []
                    for g in range(len(vars(ds[i].fields['arg_idx'])['array'])):
                        if int(vars(ds[i].fields['arg_idx'])['array'][g][0]) == config.max_seq_len or int(vars(ds[i].fields['arg_idx'])['array'][g][1]) == config.max_seq_len:
                            if int(vars(ds[i].fields['arg_idx'])['array'][g][0]) == config.max_seq_len:
                                ent1.append(vars(ds[i].fields['tokens'])['tokens'][g-1])
                            else:
                                ent2.append(vars(ds[i].fields['tokens'])['tokens'][g-1])
    
                    tostr = lambda a: [str(a[i]) for i in range(len(a))] 
                    writer.writerow([" ".join(tostr(vars(ds[i].fields['tokens'])['tokens'])), [ent1, ent2], pred[i], true[i], i])
                if with_element == 1:
                    writer.writerow("")

In [12]:

if __name__ == '__main__':

    tacred_reader = DataLoader(train_path, list(e_label2idx.keys()), list(r_label2idx.keys()), {"lower": True, "len": config.max_seq_len})
    
    token_indexer = PretrainedBertIndexer(
        pretrained_model="bert-base-uncased",
#         do_lowercase=False               # for cased condition
    )
 
	# AllenNLP DatasetReader
    reader = RelationDatasetReader(
        is_training=True, 
        reader=tacred_reader, 
        token_indexers={"tokens": token_indexer}
    )

    train_ds = reader.read(train_path)
    print(len(train_ds))
#     print(type(train_ds[0]))
#     for e in range(20):
#         print(len(vars(train_ds[e].fields['tokens'])['tokens']))
#         print(vars(train_ds[e].fields['tokens']))
#         print(len(vars(train_ds[0].fields['arg_idx'])['array']))
#         print(vars(train_ds[e].fields['arg_idx']))
#         print(vars(train_ds[e].fields['sen_len']))
#         print(vars(train_ds[e].fields['label']))
    
    # user-defined new label
    vocab = Vocabulary()

    iterator = BucketIterator(batch_size=config.batch_size, sorting_keys=[("tokens", "num_tokens")])
    iterator.index_with(vocab)

    bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-uncased",
        top_layer_only=True, # conserve memory   
    )
    
    word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                                # we'll be ignoring masks so we'll need to set this to True
                                                               allow_unmatched_keys = True)

#     print(token_indexer.tokens_to_indices([Token(text="[unused100]")], vocab, "test"))

    model = BERT(word_embeddings)
    if USE_GPU:
        model.cuda()

    optimizer = optim.Adam(model.parameters(), lr=config.lr)

100%|██████████| 22631/22631 [00:44<00:00, 504.49it/s]
22631it [00:22, 1016.72it/s]


22631


In [13]:
    # training
    from allennlp.training.trainer import Trainer

    trainer = Trainer(
        model=model,
        optimizer=optimizer,
        iterator=iterator,
        train_dataset=train_ds,
        cuda_device=0 if USE_GPU else -1,
        num_epochs=config.epochs,
    )

In [14]:
    # train the model 
    metrics = trainer.train()

  0%|          | 0/114 [00:00<?, ?it/s]

tensor(24, device='cuda:0')


loss: 3.7134 ||:   1%|          | 1/114 [00:10<18:51, 10.01s/it]

tensor([-0.2518,  0.1652, -0.0495, -0.0221,  0.8059,  1.1774,  0.0598,  0.4681,
         0.5558,  0.6295, -0.5866,  0.8782, -0.1160, -0.6415,  0.6031,  0.1350,
        -0.3219,  1.3848,  0.4577,  0.0767,  0.2865,  0.4113, -0.1523,  0.2720,
        -0.0000, -0.0000,  0.0000, -0.0000, -0.0000], device='cuda:0')
tensor(55, device='cuda:0')


loss: 3.6259 ||:   2%|▏         | 2/114 [00:13<15:13,  8.16s/it]

...

loss: 2.0802 ||:  21%|██        | 24/114 [01:14<04:38,  3.10s/it]

tensor([-0.3752,  0.2348, -0.2214,  0.5285,  0.7529, -0.5713,  0.1720, -0.0688,
        -0.2315, -0.1011, -1.3751, -0.1801, -0.5515, -0.3268, -0.9099, -1.0234,
        -0.2033, -0.0620,  0.3569, -0.2109, -0.1290,  0.0503, -0.5997, -0.1492,
        -0.0138,  0.7186, -0.0550,  0.2298,  0.3692, -0.9238, -0.1801, -0.2178,
         0.1637,  0.2056, -0.1512, -0.2972, -0.3882, -0.2018, -0.2126, -0.2460,
         0.6640, -0.9931, -0.7504, -0.5517, -0.4823, -0.8655,  0.0893,  0.4196,
        -0.1761, -0.4303, -0.5311, -0.1817, -0.5809, -0.2736, -0.4974, -0.3323,
        -0.1678, -0.9626, -0.1337, -0.4503,  0.3740,  0.7161, -0.1206, -1.0261,
        -0.7538, -0.7946,  0.3036, -0.0000, -0.0000, -0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000,
        -0.0000, -0.0000, -0.0000, -0.0000, -0.0000, -0.0000], device='cuda:0')
tensor(60, device='cuda:0')


loss: 2.0538 ||:  22%|██▏       | 25/114 [01:18<04:49,  3.26s/it]

tensor([-0.3727,  0.0238, -0.4031, -0.3669, -0.2611, -0.1235, -0.5400,  0.3754,
        -0.5285, -1.0664, -0.0659,  0.3450, -0.3056, -0.2558, -0.5310, -0.9730,
        -0.4710, -0.3891, -0.9537, -0.9231, -0.5030,  0.0388,  0.2558, -0.3295,
        -0.5539,  0.4796, -1.1675, -0.2186,  0.2746, -0.5447, -1.1008, -0.5850,
        -1.0412, -1.1223,  0.3115, -0.0717,  0.6108, -0.3918,  0.7965, -0.5900,
        -0.6123, -0.7932,  0.0574, -0.0054, -0.5268, -0.4189, -0.2576, -0.6752,
         0.2362, -0.3860, -0.6212,  0.2131, -0.3184, -0.5209, -0.0695, -0.6485,
        -0.2603,  0.1662,  0.8775,  0.8537, -0.0000,  0.0000, -0.0000,  0.0000,
        -0.0000, -0.0000], device='cuda:0')
tensor(34, device='cuda:0')


KeyboardInterrupt: 

In [ ]:
    # load model
#     model.load_state_dict(T.load(model_folder + "model.th"))

In [ ]:
    # save 
    with open(model_folder+'model.th', 'wb') as f:
        T.save(model.state_dict(), f)

In [ ]:
    # training data analysis
    seq_iterator = BasicIterator(batch_size=config.batch_size)
    seq_iterator.index_with(vocab)
    
    predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
    train_preds = predictor.predict(train_ds) 
    
    label_types = [r_idx2label.get(i.fields['label'].label) for i in train_ds]
    predict_types = [r_idx2label.get(i) for i in np.argmax(train_preds, axis=-1)]
    err_analyze(train_ds, label_types, predict_types, "train")

In [ ]:
    plot_comfusion_matrix(label_types, predict_types, output_path, "train_full")

In [ ]:
    # testing data analysis
    tacred_reader = DataLoader(test_path, list(e_label2idx.keys()), list(r_label2idx.keys()), {"lower": True, "len": config.max_seq_len})
    
    # AllenNLP DatasetReader
    reader = RelationDatasetReader(
        is_training=True, 
        reader=tacred_reader, 
        token_indexers={"tokens": token_indexer}
    )
    
    test_ds = reader.read(test_path)
    print(len(test_ds))
    seq_iterator = BasicIterator(batch_size=config.batch_size)
    seq_iterator.index_with(vocab)
    
    predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
    test_preds = predictor.predict(test_ds) 
    
    label_types = [r_idx2label.get(i.fields['label'].label) for i in test_ds]
    predict_types = [r_idx2label.get(i) for i in np.argmax(test_preds, axis=-1)]  
    
    err_analyze(test_ds, label_types, predict_types, "test")

In [ ]:
    plot_comfusion_matrix(label_types, predict_types, output_path, "test_full")